In [2]:
!pip install openai==0.28 pinecone-client pandas pyproj

import os
import pandas as pd
import openai
from pyproj import Transformer
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
openai.api_key = os.environ['OPENAI_API_KEY']

realtor_data = pd.read_csv('realtor-data.csv')
public_schools = pd.read_csv('Public_Schools.csv')
private_schools = pd.read_csv('Private_Schools.csv')

# Sample data as original dataset too large
realtor_sample = realtor_data.sample(n=10000, random_state=42)
public_schools_sample = public_schools.sample(n=5000, random_state=42)
private_schools_sample = private_schools.sample(n=5000, random_state=42)

print("Realtor Data Shape:", realtor_sample.shape)
print("Public Schools Data Shape:", public_schools_sample.shape)
print("Private Schools Data Shape:", private_schools_sample.shape)

print("\nRealtor Data Sample:")
print(realtor_sample.head())
print("\nPublic Schools Data Sample:")
print(public_schools_sample.head())
print("\nPrivate Schools Data Sample:")
print(private_schools_sample.head())




Realtor Data Shape: (10000, 12)
Public Schools Data Shape: (5000, 33)
Private Schools Data Shape: (5000, 33)

Realtor Data Sample:
         brokered_by    status     price  bed  bath  acre_lot     street  \
1696936      54239.0      sold  275000.0  1.0   1.0       NaN  1617038.0   
2092671      90564.0      sold  399900.0  1.0   1.0       NaN  1497499.0   
742044       53271.0  for_sale   75000.0  NaN   NaN      2.25  1877529.0   
1424136      12926.0      sold  325000.0  3.0   2.0      0.09   892999.0   
812329       79221.0  for_sale  169900.0  NaN   NaN      3.70  1998116.0   

                    city          state  zip_code  house_size prev_sold_date  
1696936            Miami        Florida   33156.0       846.0     2022-02-28  
2092671        San Diego     California   92108.0       667.0     2022-04-28  
742044   Oceola Township       Michigan   48855.0         NaN            NaN  
1424136        Worcester  Massachusetts    1603.0      1409.0     2021-11-29  
812329           

In [ ]:
# Define reqiuired fields
realtor_required_fields = ['price', 'bed', 'bath', 'city', 'state', 'zip_code', 'street']
public_schools_required_fields = ['NAME', 'CITY', 'STATE', 'ZIP', 'X', 'Y']
private_schools_required_fields = ['NAME', 'CITY', 'STATE', 'ZIP', 'X', 'Y']

# Fix ZIP Code format
realtor_sample['zip_code'] = realtor_sample['zip_code'].apply(
    lambda x: f"{int(x):05}" if pd.notnull(x) else "UNKNOWN"
)

# Clean realtor data
realtor_clean = realtor_sample.dropna(subset=realtor_required_fields)
realtor_clean = realtor_clean[
    (realtor_clean['price'] > 0) &
    (realtor_clean['bed'] > 0) &
    (realtor_clean['bath'] > 0)
]

print("\nCleaned Realtor Data Shape:", realtor_clean.shape)
print("Cleaned Realtor Data Sample:")
print(realtor_clean.head())

# Clean Public Schools Data
public_schools_clean = public_schools_sample.dropna(subset=public_schools_required_fields)
print("\nCleaned Public Schools Data Shape:", public_schools_clean.shape)
print("Cleaned Public Schools Data Sample:")
print(public_schools_clean.head())

# Clean Private Schools Data
private_schools_clean = private_schools_sample.dropna(subset=private_schools_required_fields)
print("\nCleaned Private Schools Data Shape:", private_schools_clean.shape)
print("Cleaned Private Schools Data Sample:")
print(private_schools_clean.head())

realtor_clean['description'] = realtor_clean.apply(
    lambda row: f"Property in {row['city']}, {row['state']}, ZIP {row['zip_code']}, priced at ${int(row['price'])}, "
                f"{int(row['bed'])} bedrooms, "
                f"{int(row['bath'])} bathrooms, "
                f"located on {row['street']}.",
    axis=1
)

public_schools_clean['description'] = public_schools_clean.apply(
    lambda row: f"Public school {row['NAME']} located in {row['CITY']}, {row['STATE']}, ZIP {int(row['ZIP'])}.",
    axis=1
)

private_schools_clean['description'] = private_schools_clean.apply(
    lambda row: f"Private school {row['NAME']} located in {row['CITY']}, {row['STATE']}, ZIP {int(row['ZIP'])}.",
    axis=1
)

print("\nRealtor Data with Description:")
print(realtor_clean[['description']].head())

print("\nPublic Schools Data with Description:")
print(public_schools_clean[['description']].head())

print("\nPrivate Schools Data with Description:")
print(private_schools_clean[['description']].head())




In [ ]:
transformer = Transformer.from_crs("epsg:3857", "epsg:4326", always_xy=True)

public_schools_clean[['longitude', 'latitude']] = public_schools_clean.apply(
    lambda row: pd.Series(transformer.transform(row['X'], row['Y'])),
    axis=1
)

print("\nPublic Schools with Latitude and Longitude:")
print(public_schools_clean[['NAME', 'CITY', 'STATE', 'ZIP', 'latitude', 'longitude']].head())

private_schools_clean[['longitude', 'latitude']] = private_schools_clean.apply(
    lambda row: pd.Series(transformer.transform(row['X'], row['Y'])),
    axis=1
)

print("\nPrivate Schools with Latitude and Longitude:")
print(private_schools_clean[['NAME', 'CITY', 'STATE', 'ZIP', 'latitude', 'longitude']].head())


def get_openai_embeddings(texts, model="text-embedding-3-small", batch_size=1000):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        try:
            response = openai.Embedding.create(input=batch, model=model)
            batch_embeddings = [data['embedding'] for data in response['data']]
            embeddings.extend(batch_embeddings)
            print(f"Processed batch {i // batch_size + 1}")
        except openai.OpenAIError as e:
            print(f"Error processing batch {i // batch_size + 1}: {e}")
    return embeddings

realtor_embeddings = get_openai_embeddings(realtor_clean['description'].tolist(), model="text-embedding-3-small")
public_schools_embeddings = get_openai_embeddings(public_schools_clean['description'].tolist(), model="text-embedding-3-small")
private_schools_embeddings = get_openai_embeddings(private_schools_clean['description'].tolist(), model="text-embedding-3-small")

print("Realtor Embeddings Length:", len(realtor_embeddings))
print("Public Schools Embeddings Length:", len(public_schools_embeddings))
print("Private Schools Embeddings Length:", len(private_schools_embeddings))



os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
# Init Pinecone
api_key = os.environ["PINECONE_API_KEY"]
pinecone = Pinecone(api_key=api_key)

index_name = "real-estate-index-1536"

indexes = [index['name'] for index in pinecone.list_indexes()]
if index_name not in indexes:
    spec = ServerlessSpec(cloud="aws", region="us-east-1")
    pinecone.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=spec
    )
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists.")


index = pinecone.Index(index_name)
print(f"Connected to Pinecone index: {index_name}")


batch_size = 100


realtor_vectors = [
    {
        "id": f"re_{idx}",
        "values": embedding,
        "metadata": {
            "price": row['price'] if pd.notnull(row['price']) else 0,
            "bed": int(row['bed']) if pd.notnull(row['bed']) else 0,
            "bath": int(row['bath']) if pd.notnull(row['bath']) else 0,
            "city": row['city'] if pd.notnull(row['city']) else "",
            "state": row['state'] if pd.notnull(row['state']) else "",
            "zip_code": row['zip_code'] if pd.notnull(row['zip_code']) else "",
            "house_size": row['house_size'] if pd.notnull(row['house_size']) else 0,
            "description": row['description'] if pd.notnull(row['description']) else ""
        }
    }
    for idx, (embedding, (_, row)) in enumerate(zip(realtor_embeddings, realtor_clean.iterrows()), start=0)
]


for i in range(0, len(realtor_vectors), batch_size):
    batch = realtor_vectors[i:i + batch_size]
    try:
        index.upsert(vectors=batch)
        print(f"Upserted Realtor batch {i // batch_size + 1}")
    except pinecone.exceptions.PineconeException as e:
        print(f"Failed to upsert Realtor batch {i // batch_size + 1}: {e}")

print("Realtor vectors upserted successfully!")

public_schools_vectors = [
    {
        "id": f"pub_{idx}",
        "values": embedding,
        "metadata": {
            "name": row['NAME'] if pd.notnull(row['NAME']) else "",
            "city": row['CITY'] if pd.notnull(row['CITY']) else "",
            "state": row['STATE'] if pd.notnull(row['STATE']) else "",
            "zip_code": row['ZIP'] if pd.notnull(row['ZIP']) else "",
            "latitude": row['latitude'] if pd.notnull(row['latitude']) else 0.0,
            "longitude": row['longitude'] if pd.notnull(row['longitude']) else 0.0,
            "description": row['description'] if pd.notnull(row['description']) else ""
        }
    }
    for idx, (embedding, (_, row)) in enumerate(zip(public_schools_embeddings, public_schools_clean.iterrows()), start=0)
]


for i in range(0, len(public_schools_vectors), batch_size):
    batch = public_schools_vectors[i:i + batch_size]
    try:
        index.upsert(vectors=batch)
        print(f"Upserted Public Schools batch {i // batch_size + 1}")
    except pinecone.exceptions.PineconeException as e:
        print(f"Failed to upsert Public Schools batch {i // batch_size + 1}: {e}")

print("Public Schools vectors upserted successfully!")


private_schools_vectors = [
    {
        "id": f"pri_{idx}",
        "values": embedding,
        "metadata": {
            "name": row['NAME'] if pd.notnull(row['NAME']) else "",
            "city": row['CITY'] if pd.notnull(row['CITY']) else "",
            "state": row['STATE'] if pd.notnull(row['STATE']) else "",
            "zip_code": row['ZIP'] if pd.notnull(row['ZIP']) else "",
            "latitude": row['latitude'] if pd.notnull(row['latitude']) else 0.0,
            "longitude": row['longitude'] if pd.notnull(row['longitude']) else 0.0,
            "description": row['description'] if pd.notnull(row['description']) else ""
        }
    }
    for idx, (embedding, (_, row)) in enumerate(zip(private_schools_embeddings, private_schools_clean.iterrows()), start=0)
]


for i in range(0, len(private_schools_vectors), batch_size):
    batch = private_schools_vectors[i:i + batch_size]
    try:
        index.upsert(vectors=batch)
        print(f"Upserted Private Schools batch {i // batch_size + 1}")
    except pinecone.exceptions.PineconeException as e:
        print(f"Failed to upsert Private Schools batch {i // batch_size + 1}: {e}")

print("Private Schools vectors upserted successfully!")



In [ ]:
def get_query_embedding(query_text):
    response = openai.Embedding.create(input=[query_text], model="text-embedding-ada-002")
    return response['data'][0]['embedding']

# Test query for house listing
query_text_house = "Looking for a 4-bedroom house in CA priced around $500000."
query_embedding_house = get_query_embedding(query_text_house)


response_house = index.query(
    vector=query_embedding_house,
    top_k=5,
    include_metadata=True
)


print("House Listing Results:")
for match in response_house['matches']:
    print(f"ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Metadata: {match['metadata']}")
    print("-" * 50)

query_text_public_school = "Looking for public schools in Los Angeles, CA."
query_embedding_public_school = get_query_embedding(query_text_public_school)

# Pinecone public school query
response_public_school = index.query(
    vector=query_embedding_public_school,
    top_k=5,
    include_metadata=True
)

# Public School
print("Public School Results:")
for match in response_public_school['matches']:
    print(f"ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Metadata: {match['metadata']}")
    print("-" * 50)


query_text_private_school = "Looking for private schools in San Francisco, CA."
query_embedding_private_school = get_query_embedding(query_text_private_school)

# Pinecone private school query
response_private_school = index.query(
    vector=query_embedding_private_school,
    top_k=5,
    include_metadata=True
)

print("Private School Results:")
for match in response_private_school['matches']:
    print(f"ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Metadata: {match['metadata']}")
    print("-" * 50)